In [3]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import import_ipynb
import twitter_credentials
import json
import csv
from tweepy.parsers import JSONParser

importing Jupyter notebook from twitter_credentials.ipynb


In [60]:
"""
    Twitter Authenticator
"""

class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

In [61]:
class TwitterStreamer():
    """
        Class for streaming and processing live tweets
    """
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
    
    # Handles Twitter authentication and connection to the Twitter Streaming API
    def stream_tweets(self, fetched_tweets_filename, hashtags):  
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_authenticator.authenticate_twitter_app()
    
        # listener responsible of how to deal with data (tweets) and error
        stream = Stream(auth, listener) 
        # filter to stream all tweets containing the word(s) in track. 
        # The track parameter is an array of search terms to stream
        stream.filter(track=hashtags)
    
    def hashtag_handler(self):
        auth = self.twitter_authenticator.authenticate_twitter_app()
        
    

In [62]:
  
# inherit from stream listener class. it has methods that we can override
"""
    Listener that prints receivedtweets to stdout
"""
class TwitterListener(StreamListener):
    
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename
        
    
    # takes in data that is streamed by the stream listener. We can do whatever we want with that data
    #The on_data method of a stream listener receives all messages and calls functions according to the message type
    def on_data(self, data):  
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as f:
                f.write(data)
            return True
        except BaseException as e:
            print('Error on_data: %s' % str(e))
        return True
    
    # overwriting. When an error occurs
    def on_error(self, status):
        if status == 420:
            print(status)
            # return false in case rate limit occurs
            return False
        print(status)

        

In [146]:
class HashtagHandler():
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
    
    def search_hashtags(self, query, max_tweets, from_date, to_date, file_name):
        auth = self.twitter_authenticator.authenticate_twitter_app()
        api = API(auth)
        
        csvFile = open(file_name, 'a')
        csvWriter = csv.writer(csvFile)
    
        for tweet in Cursor(api.search, 
                     q=query, count=max_tweets, since=from_date, until=to_date,   
                     result_type="mixed", lang="en").items(max_tweets):
            if not tweet.retweeted and 'RT @' not in tweet.text:
                csvWriter.writerow([tweet.id, tweet.created_at, tweet.text, len(tweet.text), 
                                    tweet.source, tweet.favorite_count, tweet.retweeted, tweet.retweet_count])
        csvFile.close()


In [147]:
if __name__ == "__main__":

    hashtags = ['krispykreme', 'KrispyKreme']
    test = HashtagHandler()
    file_name = 'krispy_tweets8.csv'
    tweets =test.search_hashtags(hashtags, 120, '2021-03-28', '2021-04-01', file_name)
    
         

In [6]:
with open('krispy_tweets8.csv', 'rU') as myfile:
    filtered = (line.replace('\r', '') for line in myfile)
    for row in csv.reader(filtered):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-6-a8d749015bcb>, line 4)

In [ ]:
csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])